### 토픽 모델링(Topic Modeling) - 20 뉴스그룹
+ 토픽 모델링: 문서 집합에 숨어 있는 주제를 찾아내는 것이다.
    + 사람이 수행하는 토픽 모델링: 더 함축적인 의미로 문장을 요약하는 것
    + 머신러닝 기반의 토픽 모델링: 숨겨진 주제를 효과적으로 표현할 수 있는 중심 단어를 함축적으로 추출하는 것
        + LSA(Latent Semantic Analysis) 기법
        + LDA(Latent Dirichlet Allocation) 기법: Count 기반의 피처 벡터화만 가능
            + LatentDirchletAllocation 클래스: LDA 기반의 토픽 모델링을 수현한 클래스


fetch_20newsgroups() API: categories 파라미터를 통해 필요한 주제만 필터링해 추출하고 추출된 텍스트를 Count 기반으로 벡터화 변환

In [2]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# 모토사이클, 야구, 그래픽스, 윈도우즈, 중동, 기독교, 의학, 우주 주제를 추출. 
cats = ['rec.motorcycles', 'rec.sport.baseball', 'comp.graphics', 'comp.windows.x',
        'talk.politics.mideast', 'soc.religion.christian', 'sci.electronics', 'sci.med'  ]

# 위에서 cats 변수로 기재된 category만 추출. featch_20newsgroups( )의 categories에 cats 입력
news_df= fetch_20newsgroups(subset='all',remove=('headers', 'footers', 'quotes'), 
                            categories=cats, random_state=0)

#LDA 는 Count기반의 Vectorizer만 적용합니다.  
count_vect = CountVectorizer(max_df=0.95, max_features=1000, min_df=2, stop_words='english', ngram_range=(1,2))
feat_vect = count_vect.fit_transform(news_df.data)
print('CountVectorizer Shape:', feat_vect.shape)

CountVectorizer Shape: (7862, 1000)


In [3]:
lda = LatentDirichletAllocation(n_components=8, random_state=0)
lda.fit(feat_vect)

C:\Users\chkwon\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=8, n_jobs=1,
             n_topics=None, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

**components_ 속성: 개별 토픽 별로 각 word 피처가 얼마나 많이 그 토픽에 할당됐는지에 대한 수치를 가지고 있다. 높은 값일수록 해당 word 피처는 그 토픽의 중심 word가 된다.** 

해당 자료는 8개의 토픽별로 1000개의 word 피처가 해당 토픽별로 연관도 값을 가지고 잇다.

In [4]:
print(lda.components_.shape)
lda.components_

(8, 1000)


array([[4.76415412e+00, 7.32035553e+01, 1.25197989e-01, ...,
        1.22458816e+02, 1.25230793e-01, 1.09228801e+02],
       [1.25128874e-01, 2.72275563e+00, 1.25032067e-01, ...,
        4.05483014e+00, 1.25201248e-01, 1.25936924e-01],
       [1.31387985e-01, 1.25043289e-01, 1.25490482e-01, ...,
        5.34009231e+01, 1.41931826e-01, 4.48156799e+01],
       ...,
       [1.25238487e-01, 1.25171127e-01, 1.25045244e-01, ...,
        2.98496263e+02, 1.25117657e-01, 1.25271339e-01],
       [2.76601555e+00, 3.84121246e+02, 1.25026823e-01, ...,
        1.25961528e+01, 2.20914848e+02, 4.50744569e+01],
       [4.89866042e+02, 8.38133615e+01, 1.85898507e+02, ...,
        4.91485772e+00, 7.17143691e+01, 1.48487732e+01]])

display_topics 함수 생성: 토픽별로 연관도가 높은 순으로 Word를 나열

In [5]:
def display_topics(model, feature_names, no_top_words):
    for topic_index, topic in enumerate(model.components_):
        print('Topic #',topic_index)

        # components_ array에서 가장 값이 큰 순으로 정렬했을 때, 그 값의 array index를 반환. 
        topic_word_indexes = topic.argsort()[::-1]
        top_indexes=topic_word_indexes[:no_top_words]
        
        # top_indexes대상인 index별로 feature_names에 해당하는 word feature 추출 후 join으로 concat
        feature_concat = ' '.join([feature_names[i] for i in top_indexes])                
        print(feature_concat)

# CountVectorizer객체내의 전체 word들의 명칭을 get_features_names( )를 통해 추출
feature_names = count_vect.get_feature_names()

# Topic별 가장 연관도가 높은 word를 15개만 추출
display_topics(lda, feature_names, 15)


Topic # 0
just time year don said like know didn bike good years got game think going
Topic # 1
image graphics jpeg dos software thanks color data images gif does format computer pc information
Topic # 2
god jesus church christ christian people believe christians bible faith sin paul man life team
Topic # 3
medical research health disease 1993 information patients cancer number hiv study use 10 april treatment
Topic # 4
file program window use server version display available windows ftp output code sun set motif
Topic # 5
don like just know think does people way make good problem want use time ve
Topic # 6
armenian people israel armenians jews turkish jewish israeli government war said turkey arab 000 armenia
Topic # 7
edu com 00 10 dos dos 12 24 pub mail cs 11 16 ac 3d 04
